In [1]:
import pandas as pd
import numpy as np
import glob
import re

In [11]:
def read_details(path='data/Details_subgroup_I_Submission.xlsx'):
    df = pd.read_excel(path,header=2)
    df['apnea'] = df['Diagnosis'].apply(lambda x: x in ['SAOS'])
    other_diagnoses = ['SRVAS','PLMS','D. Afectiva','EPILEPSIA','S. PERNAS INQUIETAS','PRIVACAO DE SONO','Parasomnia','RONCOPATIA']
    for col in other_diagnoses:
        df[col] = df.Diagnosis.apply(lambda x: x == col)
    levels = ['W%','N1%','N2%','N3%','REM%']
    for col in levels:
        meanval = df[col] + df[col+'.1']
        df[col] = meanval.values/2
    return df[['Subject','Age','Sex','apnea']+levels+other_diagnoses+['Epoches']]

def smooth_df(df,pid=-1,print_threshold = 4):
    newepochs = []
    newstages = []
    newspo2 = []
    newhr = []
    min_epoch = 10000
    for i, row in df.sort_values('Epoch').iterrows():
        #skip missing or impossible values, input later
        if row.SpO2 < 50 or row.HR < 30 or row.SpO2 > 100 or row.HR > 150:
            continue
        #don't start counting epochs until a reading is not zero, happens often
        min_epoch = min(min_epoch,row.Epoch)

        #if we skip an epoch take the mean of intermediate values. For sleep stage take the closest value
        if len(newepochs) > 0 and row.Epoch - newepochs[-1] > 1:
            #print stuff if we're missing a lot of stuff
            if row.Epoch - newepochs[-1] > print_threshold:
                print(pid,':',row.Epoch, newepochs[-1])
            inputed_epoch = newepochs[-1] + 1
            meanspo2 = (newspo2[-1] + row.SpO2)/2
            meanhr = (newhr[-1] + row.HR)/2
            while inputed_epoch < row.Epoch:
                newepochs.append(inputed_epoch)
                newspo2.append(meanspo2)
                newhr.append(meanhr)
                if inputed_epoch - newepochs[-1] < row.Epoch - inputed_epoch:
                    newstages.append(newstages[-1])
                else:
                    newstages.append(row.Stage)
                inputed_epoch = inputed_epoch + 1
        
        newepochs.append(row.Epoch)
        newstages.append(row.Stage)
        newspo2.append(row.SpO2)
        newhr.append(row.HR)
    return pd.DataFrame({"Epoch":[i - min_epoch for i in newepochs],"Stage":newstages,"SpO2":newspo2,"HR":newhr})

def read_patients(folder='data/IRSRUC_Subgroup1',details_file='data/Details_subgroup_I_Submission.xlsx',offset=0):
    details = read_details(details_file).set_index('Subject')
    subfolders = glob.glob(folder+'/*')
    res = [read_patient(sf,parent=folder) for sf in subfolders]
    stuff = []
    for pid,dflist in res:
        #todo: add check for multiple rows
        data = details.loc[pid].to_dict()
        data['patient_id'] = pid
        for df in dflist:
            entry = {k:v for k,v in data.items()}
            for col in df.columns:
                entry[col] = df[col].values
            stuff.append(entry)
    return pd.DataFrame(stuff).set_index('patient_id')

def read_patient(folder,parent):
    pid = int(folder.replace(parent,'').replace('/',''))
    files = glob.glob(folder+'/'+str(pid)+'*.xlsx')
    res = []
    for file in files:
        df = pd.read_excel(file,usecols=[0,1,2,3,4,5])
        try:
            df = df.dropna(axis=0,how='any',subset=['Epoch'])
            #they merged the event and hr columns a lot so take them both and extract the first number in there
            df.HR = (df.HR.fillna('').astype(str) + df.Events.fillna('').astype(str)).apply(lambda x: int(re.findall(r'\d+',x)[0]) if isinstance(x,str) else x).astype(int)
            df.SpO2 = df.SpO2.apply(lambda x: int(re.findall(r'\d+',x)[0]) if isinstance(x,str) else x).astype(int)
            df = df.dropna(axis=0,how='any',subset=['SpO2','HR'])
            df = smooth_df(df,pid)
            res.append(df)
        except Exception as e:
            print(e)
            print(pid,file,df.columns)
        
    return pid,res

In [12]:
subgroup1 = read_patients()
control = read_patients('data/ISRUC_Subgroup3','data/Details_subgroup_III_Submission.xlsx')
control.head()

7 : 32 2
7 : 45 34
7 : 207 201
7 : 223 217
7 : 237 223
7 : 272 264
7 : 32 2
7 : 45 34
7 : 207 201
7 : 223 217
7 : 237 223
7 : 272 264
95 : 325 314
95 : 789 784
95 : 325 314
95 : 789 784
16 : 215 210
16 : 225 219
16 : 418 412
16 : 777 772
16 : 792 782
16 : 809 804
16 : 821 815
16 : 836 826
25 : 22 17
25 : 38 22
25 : 22 17
25 : 38 22
92 : 101 96
92 : 384 376
92 : 101 96
92 : 384 376
73 : 49 44
73 : 374 369
73 : 396 388
73 : 469 464
73 : 588 580
73 : 789 778
73 : 49 44
73 : 374 369
73 : 396 388
73 : 469 464
73 : 588 580
73 : 789 778
81 : 50 44
81 : 78 72
81 : 194 188
81 : 50 44
81 : 78 72
81 : 194 188
85 : 13 7
85 : 29 23
85 : 13 7
85 : 29 23
57 : 186 160
57 : 203 197
57 : 218 209
57 : 226 220
57 : 186 160
57 : 203 197
57 : 218 209
57 : 226 220
6 : 866.0 776.0
6 : 866 776
['Epoch']
11 data/IRSRUC_Subgroup1/11/11_1.xlsx Index([1, 'W', 0, '193 MChg', 'Unnamed: 4', 'B'], dtype='object')
['Epoch']
11 data/IRSRUC_Subgroup1/11/11_2.xlsx Index([1, 'U', 0, '193', ' MChg', 'B'], dtype='object')
22

,Age,Sex,apnea,W%,N1%,N2%,N3%,REM%,SRVAS,PLMS,...,EPILEPSIA,S. PERNAS INQUIETAS,PRIVACAO DE SONO,Parasomnia,RONCOPATIA,Epoches,Epoch,Stage,SpO2,HR
patient_id,,,,,,,,,,,,,,,,,,,,,
7,31,M,False,25.920,5.220,25.74,30.59,12.53,False,False,...,False,False,False,False,False,814,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[95.0, 95.0, 96.0, 95.0, 95.0, 95.0, 95.5, 95....","[75.0, 78.0, 75.0, 74.0, 73.0, 84.0, 77.0, 77...."
7,31,M,False,25.920,5.220,25.74,30.59,12.53,False,False,...,False,False,False,False,False,814,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[95.0, 95.0, 96.0, 95.0, 95.0, 95.0, 95.5, 95....","[75.0, 78.0, 75.0, 74.0, 73.0, 84.0, 77.0, 77...."
3,52,M,False,10.375,8.375,31.13,36.71,13.41,False,False,...,False,False,False,False,False,824,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, N1, N2, N...","[97.0, 98.0, 96.0, 95.0, 96.0, 97.0, 95.0, 95....","[66.0, 57.0, 58.0, 64.0, 70.0, 61.0, 60.0, 74...."
3,52,M,False,10.375,8.375,31.13,36.71,13.41,False,False,...,False,False,False,False,False,824,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, N1, N2, N...","[97.0, 98.0, 96.0, 95.0, 96.0, 97.0, 95.0, 95....","[66.0, 57.0, 58.0, 64.0, 70.0, 61.0, 60.0, 74...."
10,33,M,False,18.470,27.760,22.74,14.07,16.96,False,False,...,False,False,False,False,False,796,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[85.0, 76.0, 78.0, 83.0, 91.0, 90.0, 81.0, 83...."


In [13]:
control['subgroup'] = 3
subgroup1['subgroup'] = 1
data = pd.concat([control.reset_index(),subgroup1.reset_index()],axis=0)
data['patient_id'] = data['patient_id'].astype(str) + '-' + data['subgroup'].astype(str)
data = data.set_index('patient_id')
data.head()

,Age,Sex,apnea,W%,N1%,N2%,N3%,REM%,SRVAS,PLMS,...,S. PERNAS INQUIETAS,PRIVACAO DE SONO,Parasomnia,RONCOPATIA,Epoches,Epoch,Stage,SpO2,HR,subgroup
patient_id,,,,,,,,,,,,,,,,,,,,,
7-3,31,M,False,25.920,5.220,25.74,30.59,12.53,False,False,...,False,False,False,False,814,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[95.0, 95.0, 96.0, 95.0, 95.0, 95.0, 95.5, 95....","[75.0, 78.0, 75.0, 74.0, 73.0, 84.0, 77.0, 77....",3
7-3,31,M,False,25.920,5.220,25.74,30.59,12.53,False,False,...,False,False,False,False,814,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[95.0, 95.0, 96.0, 95.0, 95.0, 95.0, 95.5, 95....","[75.0, 78.0, 75.0, 74.0, 73.0, 84.0, 77.0, 77....",3
3-3,52,M,False,10.375,8.375,31.13,36.71,13.41,False,False,...,False,False,False,False,824,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, N1, N2, N...","[97.0, 98.0, 96.0, 95.0, 96.0, 97.0, 95.0, 95....","[66.0, 57.0, 58.0, 64.0, 70.0, 61.0, 60.0, 74....",3
3-3,52,M,False,10.375,8.375,31.13,36.71,13.41,False,False,...,False,False,False,False,824,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, N1, N2, N...","[97.0, 98.0, 96.0, 95.0, 96.0, 97.0, 95.0, 95....","[66.0, 57.0, 58.0, 64.0, 70.0, 61.0, 60.0, 74....",3
10-3,33,M,False,18.470,27.760,22.74,14.07,16.96,False,False,...,False,False,False,False,796,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[85.0, 76.0, 78.0, 83.0, 91.0, 90.0, 81.0, 83....",3


In [14]:
data.apnea.astype(int).sum(), data.shape[0] - data.apnea.sum()

(121, 94)

In [15]:
import pickle
with open('processed_patients.pickle','wb') as f:
    pickle.dump(data,f)
with open('processed_patients.pickle','rb') as f:
    test = pickle.load(f)
test.head()

,Age,Sex,apnea,W%,N1%,N2%,N3%,REM%,SRVAS,PLMS,...,S. PERNAS INQUIETAS,PRIVACAO DE SONO,Parasomnia,RONCOPATIA,Epoches,Epoch,Stage,SpO2,HR,subgroup
patient_id,,,,,,,,,,,,,,,,,,,,,
7-3,31,M,False,25.920,5.220,25.74,30.59,12.53,False,False,...,False,False,False,False,814,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[95.0, 95.0, 96.0, 95.0, 95.0, 95.0, 95.5, 95....","[75.0, 78.0, 75.0, 74.0, 73.0, 84.0, 77.0, 77....",3
7-3,31,M,False,25.920,5.220,25.74,30.59,12.53,False,False,...,False,False,False,False,814,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[95.0, 95.0, 96.0, 95.0, 95.0, 95.0, 95.5, 95....","[75.0, 78.0, 75.0, 74.0, 73.0, 84.0, 77.0, 77....",3
3-3,52,M,False,10.375,8.375,31.13,36.71,13.41,False,False,...,False,False,False,False,824,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, N1, N2, N...","[97.0, 98.0, 96.0, 95.0, 96.0, 97.0, 95.0, 95....","[66.0, 57.0, 58.0, 64.0, 70.0, 61.0, 60.0, 74....",3
3-3,52,M,False,10.375,8.375,31.13,36.71,13.41,False,False,...,False,False,False,False,824,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, N1, N2, N...","[97.0, 98.0, 96.0, 95.0, 96.0, 97.0, 95.0, 95....","[66.0, 57.0, 58.0, 64.0, 70.0, 61.0, 60.0, 74....",3
10-3,33,M,False,18.470,27.760,22.74,14.07,16.96,False,False,...,False,False,False,False,796,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[W, W, W, W, W, W, W, W, W, W, W, W, W, W, W, ...","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[85.0, 76.0, 78.0, 83.0, 91.0, 90.0, 81.0, 83....",3
